# Demonstrate Seq2Seq Wrapper with twitter chat log

In [19]:
import tensorflow as tf
import numpy as np

# preprocessed data
from datasets.twitter import data
import data_utils

In [20]:
# load data from pickle and npy files
metadata, idx_q, idx_a = data.load_data(PATH='datasets/twitter/')
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_q, idx_a)

In [21]:
# parameters 
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 16
xvocab_size = len(metadata['idx2w'])  
yvocab_size = xvocab_size
emb_dim = 1024

In [22]:
import seq2seq_wrapper

In [23]:
import importlib
importlib.reload(seq2seq_wrapper)

<module 'seq2seq_wrapper' from '/Users/vladshostak/Desktop/tf-v1-venv/practical_seq2seq-master/seq2seq_wrapper.py'>

In [24]:
model = seq2seq_wrapper.Seq2Seq(xseq_len=xseq_len,
                               yseq_len=yseq_len,
                               xvocab_size=xvocab_size,
                               yvocab_size=yvocab_size,
                               ckpt_path='ckpt/twitter/',
                               emb_dim=emb_dim,
                               num_layers=3
                               )

<log> Started Building Graph... COMPLETE </log>

In [25]:
val_batch_gen = data_utils.rand_batch_gen(validX, validY, 256)
test_batch_gen = data_utils.rand_batch_gen(testX, testY, 256)
train_batch_gen = data_utils.rand_batch_gen(trainX, trainY, batch_size)

In [119]:
sess = model.train(train_batch_gen, val_batch_gen)


<log> Training started </log>


In [26]:
sess = model.restore_last_session()

In [27]:
input_ = test_batch_gen.__next__()[0]
output = model.predict(sess, input_)
print(output.shape)

(256, 20)


In [28]:
replies = []
for ii, oi in zip(input_.T, output):
    q = data_utils.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded = data_utils.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split(' ')
    if decoded.count('unk') == 0:
        if decoded not in replies:
            print('q : [{0}]; a : [{1}]'.format(q, ' '.join(decoded)))
            replies.append(decoded)

q : [lol tomorrow but i will read]; a : [let me know if you can get a chance to be a good person to do]
q : [but i hate meetings ]; a : [they dont even know what to say]
q : [can we just move trump to mars]; a : [if you can get a good time]
q : [is that a unk for your penis jeff for you must be warned that some of us might like that]; a : [i dont know that but i am not sure about it]
q : [i think should promote s more its crucial voters know both sides before voting in november]; a : [is the truth in the debate and trump is a liar for humanity]
q : [the answer that best unk how badly trump was out of his depth in the debate]; a : [the only thing is a joke of the debate]
q : [yeah shes from unk unk]; a : [hes not a fan of the guy]
q : [i was up here really trynna use my context unk to find out what department he was talking about]; a : [i dont think the same thing was that i just have to be able to be a fan]
q : [trump still won the vast majority of the polls and the cnn poll was rigged

In [7]:
import tensorflow as tf; print(tf.VERSION)

1.4.0
